In [ ]:
import pandas as pd
CA = pd.read_csv('mapdataall.csv')

In [ ]:
CA.shape

In [ ]:
df = pd.read_fwf('national_county.txt')
df.to_csv('national_county.csv')

In [ ]:
df = CA[['incident_acres_burned','incident_county','incident_date_extinguished', 'incident_containment', 'incident_name']]

In [ ]:
df.shape

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Exploding a Column into multiple rows

In [ ]:
type(df.incident_county[0])

In [ ]:
df = df[pd.notnull(df['incident_county'])]

In [ ]:
df['incident_date_extinguished'] = df['incident_date_extinguished'].fillna('Active')

In [ ]:
df.isnull().sum()

In [ ]:
new_df = pd.DataFrame(df.incident_county.str.split(', ').tolist(), index=[df.incident_acres_burned, df.incident_date_extinguished, df.incident_containment, df.incident_name]).stack()

In [ ]:
new_df = new_df.reset_index([0, 'incident_acres_burned', 'incident_date_extinguished','incident_containment','incident_name'])

In [ ]:
new_df.rename(columns = {0:'incident_county'}, inplace = True)

In [ ]:
new_df['incident_county'] = new_df['incident_county'] + " County"

In [ ]:
new_df['STNAME'] = 'CA'

In [ ]:
new_df.shape

In [ ]:
test = pd.read_csv('national_county.csv', header = None)

In [ ]:
test.rename(columns = {0: "x", 1 : "y"}, inplace = True)

In [ ]:
test2 = pd.DataFrame(test.y.str.split(',').tolist())


In [ ]:
test2.rename(columns = {0: "STNAME", 1 : "x", 2 : "y", 3 : "incident_county"}, inplace = True)

In [ ]:
test2.drop([4], axis = 1 ,inplace = True)

In [ ]:
test2["FIPS"] = test2['x'] + test2['y']

In [ ]:
test2.drop(['x', 'y'], axis = 1, inplace = True)

In [ ]:
testCA = test2[test2["STNAME"] == 'CA']

In [ ]:
new_df['FIPS'] = new_df.incident_county.map(testCA.set_index('incident_county')['FIPS'].to_dict())

In [ ]:
import numpy as np

In [ ]:
subset = new_df.groupby(['incident_county','FIPS']).agg({'incident_acres_burned':sum, 'incident_containment':np.mean}).reset_index()

In [ ]:
subset.shape

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


fig = px.choropleth_mapbox(subset, geojson=counties, locations='FIPS', color= 'incident_acres_burned',
                           color_continuous_scale="inferno_r",
                           hover_name = 'incident_county',
                           mapbox_style="carto-positron",
                           zoom=4.8, center = {"lat": 37.3, "lon": -119.4179},
                           opacity=0.5,
                           labels={'incident_acres_burned':'Acres Burned', 'FIPS' : 'County Code'},
                           title = 'WildFire Damage (Acres Burned) Across CA Counties'
                          )

fig.write_html("firemap2.html")
fig.show()


In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import plotly.express as px

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)


fig = px.choropleth_mapbox(subset, geojson=counties, locations='FIPS', color= 'incident_containment',
                           color_continuous_scale="Plasma_r",
                           hover_name = 'incident_county',
                           mapbox_style="carto-positron",
                           zoom=4.8, center = {"lat": 37.3, "lon": -119.4179},
                           opacity=0.5,
                           labels={'incident_containment':'Percent Contained', 'FIPS' : 'County Code'},
                           title = 'Average Wildfire Percent Contained (By County)'
                          )

fig.write_html("firemap3.html")
fig.show()


In [ ]:
burned_by_county = subset.nlargest(10, 'incident_acres_burned')

In [ ]:
import plotly.express as px

fig = px.bar(burned_by_county, y="incident_county", x="incident_acres_burned", color="incident_county", orientation="h", 
             hover_name="incident_county", labels = {'incident_county' : 'County', 'incident_acres_burned' : 'Acres Burned'},
             color_discrete_sequence = px.colors.sequential.matter_r
            )

fig.show()

In [ ]:
duplicate = df.copy()

In [ ]:
duplicate['incident_name'] = df['incident_name'].copy().str.split('(').str[0]

In [ ]:
burned_by_name = duplicate.nlargest(10, 'incident_acres_burned')

In [ ]:
import plotly.express as px

fig = px.bar(burned_by_name, y="incident_name", x="incident_acres_burned", color="incident_name", orientation="h", 
             hover_name="incident_name", hover_data = ['incident_county','incident_date_extinguished'], labels = {'incident_date_extinguished' : 'Date Extinguished', 'incident_name' : 'Incident Name', 'incident_acres_burned' : 'Acres Burned', 'incident_county' : 'County'},
             color_discrete_sequence = px.colors.sequential.matter_r
            )

fig.show()